# Création du Dataframe + initialisation de la Data

In [14]:
import pandas as pd
import json
from numpyencoder import NumpyEncoder

# Read csv file
Df = pd.read_csv('planningTestV2.csv', sep=';')
# Filter columns and rows
Df.drop(columns=Df.columns[-4:], inplace=True)
Df.reset_index()
Df.fillna(0, inplace=True)
Df

,LUNDI,MARDI,MERCREDI,JEUDI,VENDREDI
0,30-janv,31-janv,01-févr,02-févr,03-févr
1,8h-12h15,8h-12h15,8h-12h15,8h-12h15,8h-12h15
2,0,0,0,CONCEP.LOGICIELLE/BIG DATA,CONCEP.LOGICIELLE/BIG DATA
3,Introduction,Introduction,Introduction,Administration Système,0
4,Administration Système,Administration Système,Administration Système,0,0
...,...,...,...,...,...
67,0,Optimisation numérique et problème inverse,T.DUPONT,0,Pas de cours
68,0,0,Salle I304,0,0
69,0,14h45 - 17h45,0,0,0
70,0,E.VAN REETH /S.MAZAURIC,0,0,0


In [15]:
# init constant
NomMajeurs = "CONCEP.LOGICIELLE/BIG DATA","ROBOTIQUE","ELECTRONIQUE ET SYST EMB","INFRA DES RESEAUX","IMAGE" 
Groupes = "Grp A","Grp B","Grp C","Grp D"

# Create Json = list of dict
def CreateJson(NomMajeurs,Groupes):
    Data = []
    for i in NomMajeurs:
            Data.append({"NomMajeurs":i,})
    return Data


Data = CreateJson(NomMajeurs, Groupes)
print(Data)

[{'NomMajeurs': 'CONCEP.LOGICIELLE/BIG DATA'}, {'NomMajeurs': 'ROBOTIQUE'}, {'NomMajeurs': 'ELECTRONIQUE ET SYST EMB'}, {'NomMajeurs': 'INFRA DES RESEAUX'}, {'NomMajeurs': 'IMAGE'}]


In [46]:
def createSchedule(Df,Data):
    DicoJour = {}
    IsAprem = False
    LstSemaine = []
    # parcour des jours pour séparer matin et aprem
    for obj in Df :
        LstMatin = []
        LstAprem = []
        LstTemp = []
        # parcour des cases non nulles 
        for case in Df[obj] :
            if case != 0 :
                LstTemp.append(case)

        # parcour des cases non nulles d'une journée pour la séparation
        for index,value in enumerate(LstTemp) :
            if index == 0 :
                # on ajoute le jour
                DicoJour["jour"] = obj +"-" + value
            else :
                if value in {'13h-14h30','13h30-17h45','13h10-18h', '13h30-15h30', '15h45-17h45', '13h30 - 14h30'}:
                  IsAprem = True
                elif value in {'8h-12h15', '8h-10h', '10h15-12h15', '8h- 10h', }:
                    IsAprem = False
                
                if IsAprem :
                    LstAprem.append(value)
                else :
                    LstMatin.append(value)

        DicoJour["Matin"] = LstMatin.copy()
        DicoJour["Aprem"] = LstAprem.copy()
        LstSemaine.append(DicoJour.copy())
    return LstSemaine

LstSemaine = createSchedule(Df,Data)
df_semaine = pd.DataFrame( LstSemaine)
df_semaine
# df_semaine.to_json('planning_semaineMatin_Aprem.json',orient='records', indent=4)

,jour,Matin,Aprem
0,LUNDI-30-janv,"[8h-12h15, Introduction, Administration Systèm...","[13h10-18h, LV, (Voir planning)]"
1,MARDI-31-janv,"[8h-12h15, Introduction, Administration Systèm...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Base..."
2,MERCREDI-01-févr,"[8h-12h15, Introduction, Administration Systèm...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Admi..."
3,JEUDI-02-févr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, Adminis...",[]
4,VENDREDI-03-févr,"[8h-12h15, CONCEP.LOGICIELLE/BIG DATA, pas de ...","[13h30-17h45, CONCEP.LOGICIELLE/BIG DATA, Base..."


In [34]:
def filtreMaj(LstSemaine,Data) :
    CoursM = [[],[],[],[],[],[]]
    CoursA = [[],[],[],[],[],[]]

    for Jour in LstSemaine :
        index = 1
        Majeur = 0
        Semaine = {}
        CoursM=[[],[],[],[],[],[],Jour["jour"]]
        CoursA=[[],[],[],[],[],[],Jour["jour"]]
        
        for Case in Jour["Matin"] :
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = 1
            elif Case == "ROBOTIQUE" :
                Majeur = 2
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = 3
            elif Case == "INFRA DES RESEAUX " :
                Majeur = 4
            elif Case == "IMAGE " :
                Majeur = 5
            CoursM[Majeur].append(Case)

        for Case in Jour["Aprem"] :
            if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                Majeur = 1
            elif Case == "ROBOTIQUE" :
                Majeur = 2
            elif Case == "ELECTRONIQUE ET SYST EMB" :
                Majeur = 3
            elif Case == "INFRA DES RESEAUX " :
                Majeur = 4
            elif Case == "IMAGE " :
                Majeur = 5
            CoursA[Majeur].append(Case)
            
        
        for Maj in Data : 
            print(CoursM[index])
            Semaine[Jour["jour"]] = {"Matin":CoursM[index],"Aprem":CoursA[index]}
            Maj["semaine"] = Semaine
            index += 1
    return Data
Data = filtreMaj(LstSemaine,Data)
Data

,LUNDI-30-janv,MARDI-31-janv,MERCREDI-01-févr,JEUDI-02-févr,VENDREDI-03-févr
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...


In [70]:
def filtreMaj(LstSemaine) :
    semaine = {}
    # parcour des jours de la semaine
    for Jour in LstSemaine :
        semaine[Jour["jour"]] = {} # creation d'une nouvelle journée dans la semaine
        for DemiJour in Jour.keys() :
            if DemiJour == "jour" : continue
            print(DemiJour)
            # init les cours de chaque majeur pour chaque demi-journée
            j= {"CONCEP.LOGICIELLE/BIG DATA":[],
            "ROBOTIQUE":[],
            "ELECTRONIQUE ET SYST EMB":[],
            "INFRA DES RESEAUX":[],
            "IMAGE":[],
            "Pour tous":[]}
            # parcour des cours de la demi-journée
            en_majeur = 0
            n = 4 # correspond aux nombre de cases sous les majeurs en général
            for Case in Jour[DemiJour] :
                if Case == "CONCEP.LOGICIELLE/BIG DATA" :
                    Majeur = "CONCEP.LOGICIELLE/BIG DATA"
                    en_majeur = n
                elif Case == "ROBOTIQUE" :
                    Majeur = "ROBOTIQUE"
                    en_majeur = n
                elif Case == "ELECTRONIQUE ET SYST EMB" :
                    Majeur = "ELECTRONIQUE ET SYST EMB"
                    en_majeur = n
                elif Case == "INFRA DES RESEAUX " :
                    Majeur = "INFRA DES RESEAUX"
                    en_majeur = n
                elif Case == "IMAGE " :
                    Majeur = "IMAGE"
                    en_majeur = n
                if en_majeur<=0: Majeur = "Pour tous"
                if en_majeur > 0 : en_majeur -= 1 
                j[Majeur].append(Case)
                # on ajoute a chaque majeur pour la demi-journée
            # on ajoute a la semaine les cours de chaque majeur du jour en cours
            semaine[Jour["jour"]][DemiJour] = j

    return semaine

Semaine = filtreMaj(LstSemaine)
print(type(Semaine))
print(Semaine.keys())
df_Semaine = pd.DataFrame(Semaine)
print(df_Semaine.columns)
df_Semaine.to_json('planning_semaineMatin_Aprem_by_major2.json', indent=4)
df_Semaine

Matin
Aprem
Matin
Aprem
Matin
Aprem
Matin
Aprem
Matin
Aprem
<class 'dict'>
dict_keys(['LUNDI-30-janv', 'MARDI-31-janv', 'MERCREDI-01-févr', 'JEUDI-02-févr', 'VENDREDI-03-févr'])
Index(['LUNDI-30-janv', 'MARDI-31-janv', 'MERCREDI-01-févr', 'JEUDI-02-févr',
       'VENDREDI-03-févr'],
      dtype='object')


,LUNDI-30-janv,MARDI-31-janv,MERCREDI-01-févr,JEUDI-02-févr,VENDREDI-03-févr
Matin,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...","{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...
Aprem,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...,"{'CONCEP.LOGICIELLE/BIG DATA': [], 'ROBOTIQUE'...",{'CONCEP.LOGICIELLE/BIG DATA': ['CONCEP.LOGICI...


In [18]:
with open('Datatest.json', 'w+') as f:
    json.dump(Data, f, indent=4, cls=NumpyEncoder)

